<a href="https://colab.research.google.com/github/farisazizy/Artificial-Intelligence/blob/machine-learning/classification_logisticregression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Import Library
%tensorflow_version 2.x

# tensor flow module
import tensorflow as tf

# matplotlib
from matplotlib import colors
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np

## Data Cleaning

In [3]:
train = pd.read_csv("kendaraan_train.csv")
test = pd.read_csv("kendaraan_test.csv")

# Melakukan drop pada baris yang memiliki nilai null
train = train.dropna()
train.drop(["id"], axis=1,  inplace=True)
test = test.dropna()
train.reset_index(drop=True, inplace=True)

print(train.head())
print(test.head())

  Jenis_Kelamin  Umur  SIM  ...  Kanal_Penjualan  Lama_Berlangganan Tertarik
0        Wanita  30.0  1.0  ...            152.0               97.0      0.0
1          Pria  48.0  1.0  ...             29.0              158.0      0.0
2        Wanita  58.0  1.0  ...            124.0               63.0      0.0
3          Pria  21.0  1.0  ...            152.0              171.0      0.0
4        Wanita  20.0  1.0  ...            160.0               31.0      0.0

[5 rows x 11 columns]
  Jenis_Kelamin  Umur  SIM  ...  Kanal_Penjualan  Lama_Berlangganan Tertarik
0        Wanita    49    1  ...               26                145        0
1          Pria    22    1  ...              152                241        0
2          Pria    24    1  ...              152                 62        0
3          Pria    46    1  ...              124                 34        0
4          Pria    35    1  ...              152                229        0

[5 rows x 11 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:
CATEGORICAL_COLUMNS = ['Jenis_Kelamin', 'Umur_Kendaraan', 'Kendaraan_Rusak']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = train[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

print(feature_columns)

[VocabularyListCategoricalColumn(key='Jenis_Kelamin', vocabulary_list=('Wanita', 'Pria'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='Umur_Kendaraan', vocabulary_list=('< 1 Tahun', '> 2 Tahun', '1-2 Tahun'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='Kendaraan_Rusak', vocabulary_list=('Tidak', 'Pernah'), dtype=tf.string, default_value=-1, num_oov_buckets=0)]


In [5]:
object_column = train.select_dtypes(['object']).columns
category_column = train.select_dtypes(['category']).columns

train[object_column]=train[object_column].apply(lambda x: x.astype('category'))

train[object_column] = train[object_column].apply(lambda x: x.cat.codes)
train[category_column] = train[category_column].apply(lambda x: x.cat.codes)

In [6]:
object_column = test.select_dtypes(['object']).columns
category_column = test.select_dtypes(['category']).columns

test[object_column] = test[object_column].apply(lambda x: x.astype('category'))

test[object_column] = test[object_column].apply(lambda x: x.cat.codes)
test[category_column] = test[category_column].apply(lambda x: x.cat.codes)

In [7]:
train.head()

,Jenis_Kelamin,Umur,SIM,Kode_Daerah,Sudah_Asuransi,Umur_Kendaraan,Kendaraan_Rusak,Premi,Kanal_Penjualan,Lama_Berlangganan,Tertarik
0,1,30.0,1.0,33.0,1.0,1,1,28029.0,152.0,97.0,0.0
1,0,48.0,1.0,39.0,0.0,2,0,25800.0,29.0,158.0,0.0
2,1,58.0,1.0,48.0,0.0,0,1,2630.0,124.0,63.0,0.0
3,0,21.0,1.0,35.0,1.0,1,1,22735.0,152.0,171.0,0.0
4,1,20.0,1.0,8.0,1.0,1,1,30786.0,160.0,31.0,0.0


In [8]:
train_y = train.pop('Tertarik').astype(int)
test_y = test.pop('Tertarik').astype(int)
train.head() # Kolom tertarik dihilangkan untuk fitur

,Jenis_Kelamin,Umur,SIM,Kode_Daerah,Sudah_Asuransi,Umur_Kendaraan,Kendaraan_Rusak,Premi,Kanal_Penjualan,Lama_Berlangganan
0,1,30.0,1.0,33.0,1.0,1,1,28029.0,152.0,97.0
1,0,48.0,1.0,39.0,0.0,2,0,25800.0,29.0,158.0
2,1,58.0,1.0,48.0,0.0,0,1,2630.0,124.0,63.0
3,0,21.0,1.0,35.0,1.0,1,1,22735.0,152.0,171.0
4,1,20.0,1.0,8.0,1.0,1,1,30786.0,160.0,31.0


# Building the Model
Setelah kita melakukan cleaning data, kita akan menyiapkan model klasifikasi.

Model yang digunakan adalah Logistic Regression. Ada tiga metode yang akan digunakan yaitu Scratch, Sklearn, dan Keras.

Akan dibandingkan hasil dari tingkat akurasi ketiga metode tersebut.

## Logistic Regression from scratch

In [19]:
# Logistic Regression
class LogitRegression() :
    def __init__( self, learning_rate, iterations ) :        
        self.learning_rate = learning_rate        
        self.iterations = iterations
          
    # Function for model training    
    def fit( self, X, Y ) :        
        # no_of_training_examples, no_of_features        
        self.m, self.n = X.shape        
        # weight initialization        
        self.W = np.zeros( self.n )        
        self.b = 0        
        self.X = X        
        self.Y = Y
          
        # gradient descent learning
                  
        for i in range( self.iterations ) :            
            self.update_weights()            
        return self
      
    # Helper function to update weights in gradient descent
      
    def update_weights( self ) :           
        A = 1 / ( 1 + np.exp( - ( self.X.dot( self.W ) + self.b ) ) )
          
        # calculate gradients        
        tmp = ( A - self.Y.T )        
        tmp = np.reshape( tmp, self.m )        
        dW = np.dot( self.X.T, tmp ) / self.m         
        db = np.sum( tmp ) / self.m 
          
        # update weights    
        self.W = self.W - self.learning_rate * dW    
        self.b = self.b - self.learning_rate * db
          
        return self
      
    # Hypothetical function  h( x ) 
      
    def predict( self, X ) :    
        Z = 1 / ( 1 + np.exp( - ( X.dot( self.W ) + self.b ) ) )        
        Y = np.where( Z > 0.5, 1, 0 )        
        return Y
  
  
# Driver code
  
def main() :
      
    # Model training    
    model = LogitRegression( learning_rate = 0.01, iterations = 1000 )
    model.fit(train, train_y)
      
    # Prediction on test set
    pred_y = model.predict( test )    
      
    # measure performance    
    correctly_classified = 0    
      
    # counter    
    count = 0    
    for count in range( np.size( pred_y ) ) :  
        
        if test_y[count] == pred_y[count] :            
            correctly_classified = correctly_classified + 1
              
        count = count + 1
          
    print( "Accuracy on test set      :  ", ( 
      correctly_classified / count ) * 100 )

  
if __name__ == "__main__" :     
    main()

Accuracy on test set      :   87.69705493398267


## Logistic Regression with Sklearn

In [10]:
import warnings
warnings.filterwarnings( "ignore" )
  
from sklearn.linear_model import LogisticRegression

In [24]:
def main() :

    model1 = LogisticRegression(max_iter=1000)    
    model1.fit(train, train_y)
        
    pred1_y = model1.predict( test )
        
    correctly_classified1 = 0
      
    # counter    
    count = 0    
    for count in range( np.size( pred1_y ) ) :  
        
        if test_y[count] == pred1_y[count] :            
            correctly_classified1 = correctly_classified1 + 1
              
        count = count + 1

    print( "Accuracy on test set by sklearn model   :  ", ( 
      correctly_classified1 / count ) * 100 )
  
if __name__ == "__main__" :     
    main()

Accuracy on test set by sklearn model   :   87.69705493398267


## Logistic Regression with Sklearn (using SGD)

In [23]:
from sklearn.linear_model import SGDClassifier

def main() :

    model1 = SGDClassifier(eta0 = 0.01, max_iter=1000) #sgd learning path = 0,01 constant
    model1.fit(train, train_y)
        
    pred1_y = model1.predict( test )
        
    correctly_classified1 = 0
      
    # counter    
    count = 0    
    for count in range( np.size( pred1_y ) ) :  
        
        if test_y[count] == pred1_y[count] :            
            correctly_classified1 = correctly_classified1 + 1
              
        count = count + 1

    print( "Accuracy on test set by sklearn model   :  ", ( 
      correctly_classified1 / count ) * 100 )
  
if __name__ == "__main__" :     
    main()

Accuracy on test set by sklearn model   :   87.07571527529964


## Logistic Regression with Keras

In [12]:
# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [13]:
y_train = to_categorical(train_y)
y_test = to_categorical(test_y)

count_classes = y_test.shape[1]
print(count_classes)

2


In [14]:
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=10))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', #sgd
              loss='categorical_crossentropy', #loss measure
              metrics=['accuracy']) #evaluation metrics

In [26]:
model.fit(train, y_train, epochs=20)

Epoch 1/20
4062/4062 [==============================] - 10s 3ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 2/20
4062/4062 [==============================] - 10s 3ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 3/20
4062/4062 [==============================] - 10s 3ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 4/20
4062/4062 [==============================] - 10s 2ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 5/20
4062/4062 [==============================] - 10s 3ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 6/20
4062/4062 [==============================] - 11s 3ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 7/20
4062/4062 [==============================] - 10s 3ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 8/20
4062/4062 [==============================] - 10s 3ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 9/20
4062/4062 [==============================] - 10s 3ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 10/20
4062/4062 [==============================] - 10s 3ms/step - l

In [16]:
pred_test= model.predict(test)
scores2 = model.evaluate(test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1] * 100, 1 - scores2[1]))    

Accuracy on test data: 87.69705295562744% 
 Error on test data: 0.12302947044372559
